# Regrid World Ocean Atlas (WOA) Data

Here, we utilize the [WOA](https://www.ncei.noaa.gov/products/world-ocean-atlas) dataset, with this example covering the most recent version, produced in 2018

In [99]:
import xarray as xr
import xesmf as xe
import cf_xarray
import numpy as np
import pop_tools
from pop_tools.grid import _compute_corners

## Read in Sample World Ocean Atlas Data

You'll see that the file convention here is `woa{year_version}_

In [133]:
xr.open_dataset('/glade/p/cgd/oce/projects/cesm2-marbl/woa2018-data-1x1/woa18_decav_t11_01.nc', decode_times=False)

<xarray.Dataset>
Dimensions:             (lat: 180, nbounds: 2, lon: 360, depth: 57, time: 1)
Coordinates:
  * lat                 (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * depth               (depth) float32 0.0 5.0 10.0 ... 1.45e+03 1.5e+03
  * time                (time) float32 382.5
Dimensions without coordinates: nbounds
Data variables: (12/13)
    crs                 int32 ...
    lat_bnds            (lat, nbounds) float32 ...
    lon_bnds            (lon, nbounds) float32 ...
    depth_bnds          (depth, nbounds) float32 ...
    climatology_bounds  (time, nbounds) float32 ...
    t_an                (time, depth, lat, lon) float32 ...
    ...                  ...
    t_dd                (time, depth, lat, lon) float64 ...
    t_sd                (time, depth, lat, lon) float32 ...
    t_se                (time, depth, lat, lon) float32 ...
    t_oa                (time, depth, lat, lon) float32 ...
    t_ma                (time, depth, lat, lon) float32 ...
    t_gp                (time, depth, lat, lon) float64 ...
Attributes: (12/49)
    Conventions:                     CF-1.6, ACDD-1.3
    title:                           World Ocean Atlas 2018 : sea_water_tempe...
    summary:                         Climatological mean temperature for the ...
    references:                      Locarnini, R. A., A. V. Mishonov, O. K. ...
    institution:                     National Centers for Environmental Infor...
    comment:                         global climatology as part of the World ...
    ...                              ...
    publisher_email:                 NCEI.info@noaa.gov
    nodc_template_version:           NODC_NetCDF_Grid_Template_v2.0
    license:                         These data are openly available to the p...
    metadata_link:                   https://www.nodc.noaa.gov/OC5/woa18/
    date_created:                    2019-07-28 
    date_modified:                   2019-07-28

In [134]:
ds = xr.open_dataset('/glade/p/cgd/oce/projects/cesm2-marbl/woa2018-data-1x1/woa18_all_A00_01.nc', decode_times=False)

In [89]:
ds

<xarray.Dataset>
Dimensions:             (lat: 180, nbounds: 2, lon: 360, depth: 102, time: 1)
Coordinates:
  * lat                 (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * depth               (depth) float32 0.0 5.0 10.0 ... 5.3e+03 5.4e+03 5.5e+03
  * time                (time) float32 8.214e+03
Dimensions without coordinates: nbounds
Data variables:
    crs                 int32 ...
    lat_bnds            (lat, nbounds) float32 ...
    lon_bnds            (lon, nbounds) float32 ...
    depth_bnds          (depth, nbounds) float32 ...
    climatology_bounds  (time, nbounds) float32 ...
    A_an                (time, depth, lat, lon) float32 ...
    A_mn                (time, depth, lat, lon) float32 ...
    A_dd                (time, depth, lat, lon) float64 ...
    A_sd                (time, depth, lat, lon) float32 ...
    A_se                (time, depth, lat, lon) float32 ...
    A_oa                (time, depth, lat, lon) float32 ...
    A_gp                (time, depth, lat, lon) float64 ...
Attributes: (12/49)
    Conventions:                     CF-1.6, ACDD-1.3
    title:                           World Ocean Atlas 2018 : Apparent_Oxygen...
    summary:                         Climatological mean Apparent Oxygen Util...
    references:                      Garcia, H. E., K. Weathers, C. R. Paver,...
    institution:                     National Centers for Environmental Infor...
    comment:                         global climatology as part of the World ...
    ...                              ...
    publisher_email:                 NCEI.info@noaa.gov
    nodc_template_version:           NODC_NetCDF_Grid_Template_v2.0
    license:                         These data are openly available to the p...
    metadata_link:                   https://www.nodc.noaa.gov/OC5/woa18/
    date_created:                    2019-07-29 
    date_modified:                   2019-07-29

## Prepare the POP grid for regridding

In [106]:
grid = pop_tools.get_grid('POP_tx0.1v3')

In [107]:
def gen_corner_calc(ds, cell_corner_lat='ULAT', cell_corner_lon='ULONG'):
    """
    Generates corner information and creates single dataset with output
    """

    cell_corner_lat = ds[cell_corner_lat]
    cell_corner_lon = ds[cell_corner_lon]
    # Use the function in pop-tools to get the grid corner information
    corn_lat, corn_lon = _compute_corners(cell_corner_lat, cell_corner_lon)

    # Make sure this returns four corner points
    assert corn_lon.shape[-1] == 4

    lon_shape, lat_shape = corn_lon[:, :, 0].shape
    out_shape = (lon_shape + 1, lat_shape + 1)

    # Generate numpy arrays to store destination lats/lons
    out_lons = np.zeros(out_shape)
    out_lats = np.zeros(out_shape)

    # Assign the northeast corner information
    out_lons[1:, 1:] = corn_lon[:, :, 0]
    out_lats[1:, 1:] = corn_lat[:, :, 0]

    # Assign the northwest corner information
    out_lons[1:, :-1] = corn_lon[:, :, 1]
    out_lats[1:, :-1] = corn_lat[:, :, 1]

    # Assign the southwest corner information
    out_lons[:-1, :-1] = corn_lon[:, :, 2]
    out_lats[:-1, :-1] = corn_lat[:, :, 2]

    # Assign the southeast corner information
    out_lons[:-1, 1:] = corn_lon[:, :, 3]
    out_lats[:-1, 1:] = corn_lat[:, :, 3]

    return out_lats, out_lons


In [108]:
lat_corners, lon_corners = gen_corner_calc(grid)

In [113]:
pop_grid = grid[['TLAT', 'TLONG', 'z_t']]

In [116]:
pop_grid = pop_grid.rename({'TLAT':'lat',
                            'TLONG':'lon'})

In [117]:
pop_grid

<xarray.Dataset>
Dimensions:  (nlat: 2400, nlon: 3600, z_t: 62)
Coordinates:
  * z_t      (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.625e+05 5.875e+05
Dimensions without coordinates: nlat, nlon
Data variables:
    lat      (nlat, nlon) float64 -78.47 -78.47 -78.47 ... 62.34 62.34 62.34
    lon      (nlat, nlon) float64 250.0 250.2 250.2 250.3 ... 250.0 250.0 250.0
Attributes:
    lateral_dims:       [2400, 3600]
    vertical_dims:      62
    vert_grid_file:     tx0.1v3_vert_grid
    horiz_grid_fname:   inputdata/ocn/pop/tx0.1v3/grid/horiz_grid_200709.ieeer8
    topography_fname:   inputdata/ocn/pop/tx0.1v3/grid/topography_20170718.ie...
    region_mask_fname:  inputdata/ocn/pop/tx0.1v3/grid/region_mask_20170718.i...
    type:               tripole
    title:              POP_tx0.1v3 grid

In [118]:
pop_grid['lon_b'] = (('nlat_b', 'nlon_b'), lon_corners)
pop_grid['lat_b'] = (('nlat_b', 'nlon_b'), lat_corners)

In [119]:
pop_grid

<xarray.Dataset>
Dimensions:  (nlat: 2400, nlon: 3600, z_t: 62, nlat_b: 2401, nlon_b: 3601)
Coordinates:
  * z_t      (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.625e+05 5.875e+05
Dimensions without coordinates: nlat, nlon, nlat_b, nlon_b
Data variables:
    lat      (nlat, nlon) float64 -78.47 -78.47 -78.47 ... 62.34 62.34 62.34
    lon      (nlat, nlon) float64 250.0 250.2 250.2 250.3 ... 250.0 250.0 250.0
    lon_b    (nlat_b, nlon_b) float64 -110.0 -109.9 -109.8 ... -110.0 -110.0
    lat_b    (nlat_b, nlon_b) float64 -78.49 -78.49 -78.49 ... 62.34 62.34 62.34
Attributes:
    lateral_dims:       [2400, 3600]
    vertical_dims:      62
    vert_grid_file:     tx0.1v3_vert_grid
    horiz_grid_fname:   inputdata/ocn/pop/tx0.1v3/grid/horiz_grid_200709.ieeer8
    topography_fname:   inputdata/ocn/pop/tx0.1v3/grid/topography_20170718.ie...
    region_mask_fname:  inputdata/ocn/pop/tx0.1v3/grid/region_mask_20170718.i...
    type:               tripole
    title:              POP_tx0.1v3 grid

## Setup the regridder

In [124]:
pop_grid

<xarray.Dataset>
Dimensions:  (nlat: 2400, nlon: 3600, z_t: 62, nlat_b: 2401, nlon_b: 3601)
Coordinates:
  * z_t      (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.625e+05 5.875e+05
Dimensions without coordinates: nlat, nlon, nlat_b, nlon_b
Data variables:
    lat      (nlat, nlon) float64 -78.47 -78.47 -78.47 ... 62.34 62.34 62.34
    lon      (nlat, nlon) float64 250.0 250.2 250.2 250.3 ... 250.0 250.0 250.0
    lon_b    (nlat_b, nlon_b) float64 -110.0 -109.9 -109.8 ... -110.0 -110.0
    lat_b    (nlat_b, nlon_b) float64 -78.49 -78.49 -78.49 ... 62.34 62.34 62.34
Attributes:
    lateral_dims:       [2400, 3600]
    vertical_dims:      62
    vert_grid_file:     tx0.1v3_vert_grid
    horiz_grid_fname:   inputdata/ocn/pop/tx0.1v3/grid/horiz_grid_200709.ieeer8
    topography_fname:   inputdata/ocn/pop/tx0.1v3/grid/topography_20170718.ie...
    region_mask_fname:  inputdata/ocn/pop/tx0.1v3/grid/region_mask_20170718.i...
    type:               tripole
    title:              POP_tx0.1v3 grid

In [126]:
woa_ds

<xarray.Dataset>
Dimensions:             (lat: 180, lat_b: 181, lon: 360, lon_b: 361, depth: 102, nbounds: 2, time: 1)
Coordinates:
  * lat                 (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * depth               (depth) float32 0.0 5.0 10.0 ... 5.3e+03 5.4e+03 5.5e+03
  * time                (time) float32 8.214e+03
Dimensions without coordinates: lat_b, lon_b, nbounds
Data variables:
    crs                 int32 -2147483647
    lat_bnds            (lat_b) float32 -90.0 -89.0 -88.0 ... 88.0 89.0 90.0
    lon_bnds            (lon_b) float32 -180.0 -179.0 -178.0 ... 179.0 180.0
    depth_bnds          (depth, nbounds) float32 0.0 2.5 ... 5.45e+03 5.5e+03
    climatology_bounds  (time, nbounds) float32 7.512e+03 8.928e+03
    A_an                (time, depth, lat, lon) float32 ...
    A_mn                (time, depth, lat, lon) float32 ...
    A_dd                (time, depth, lat, lon) float64 ...
    A_sd                (time, depth, lat, lon) float32 ...
    A_se                (time, depth, lat, lon) float32 ...
    A_oa                (time, depth, lat, lon) float32 ...
    A_gp                (time, depth, lat, lon) float64 ...
Attributes: (12/49)
    Conventions:                     CF-1.6, ACDD-1.3
    title:                           World Ocean Atlas 2018 : Apparent_Oxygen...
    summary:                         Climatological mean Apparent Oxygen Util...
    references:                      Garcia, H. E., K. Weathers, C. R. Paver,...
    institution:                     National Centers for Environmental Infor...
    comment:                         global climatology as part of the World ...
    ...                              ...
    publisher_email:                 NCEI.info@noaa.gov
    nodc_template_version:           NODC_NetCDF_Grid_Template_v2.0
    license:                         These data are openly available to the p...
    metadata_link:                   https://www.nodc.noaa.gov/OC5/woa18/
    date_created:                    2019-07-29 
    date_modified:                   2019-07-29

In [132]:
ds.lon_bnds.shape

(361,)

In [137]:
regrid = xe.Regridder(ds, pop_grid, method='conservative')

In [138]:
regrid.to_netcdf('woa_tx0.1v3_conservative.nc')

'woa_tx0.1v3_conservative.nc'

In [153]:
xr.set_options(keep_attrs=True)

In [154]:
woa_regrid_pop = regrid(ds)

/glade/work/mgrover/miniconda3/envs/obs-validation-dev/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [157]:
woa_regrid_pop.attrs = ds.attrs
for var in woa_regrid_pop:
    woa_regrid_pop[var].attrs = ds[var].attrs

In [164]:
def convert_meters_to_cm(ds):
    """
    Converts from meters to centimeters
    """
    ds = ds.rename({'depth':'z_t'})
    ds['z_t'] = ds.z_t / 100
    return ds

In [158]:
woa_regrid_pop

<xarray.Dataset>
Dimensions:  (time: 1, depth: 102, nlat: 2400, nlon: 3600)
Coordinates:
  * depth    (depth) float32 0.0 5.0 10.0 15.0 ... 5.3e+03 5.4e+03 5.5e+03
  * time     (time) float32 8.214e+03
    lon      (nlat, nlon) float64 250.0 250.2 250.2 250.3 ... 250.0 250.0 250.0
    lat      (nlat, nlon) float64 -78.47 -78.47 -78.47 ... 62.34 62.34 62.34
Dimensions without coordinates: nlat, nlon
Data variables:
    A_an     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    A_mn     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    A_dd     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    A_sd     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    A_se     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    A_oa     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    A_gp     (time, depth, nlat, nlon) float64 nan nan nan nan ... nan nan nan
Attributes: (12/49)
    Conventions:                     CF-1.6, ACDD-1.3
    title:                           World Ocean Atlas 2018 : Apparent_Oxygen...
    summary:                         Climatological mean Apparent Oxygen Util...
    references:                      Garcia, H. E., K. Weathers, C. R. Paver,...
    institution:                     National Centers for Environmental Infor...
    comment:                         global climatology as part of the World ...
    ...                              ...
    publisher_email:                 NCEI.info@noaa.gov
    nodc_template_version:           NODC_NetCDF_Grid_Template_v2.0
    license:                         These data are openly available to the p...
    metadata_link:                   https://www.nodc.noaa.gov/OC5/woa18/
    date_created:                    2019-07-29 
    date_modified:                   2019-07-29